# Tensorflow2.0 小练习

In [1]:
import numpy as np
import tensorflow as tf

@tf.function
def func(x):
    x_exp = tf.math.exp(x)
    print(x_exp)
    out = x_exp+1
    return out

In [2]:
func(tf.constant(0.1, dtype=tf.float32))
func(tf.constant(1., dtype=tf.float32))
print()

Tensor("Exp:0", shape=(), dtype=float32)



In [113]:
import tensorflow as tf
import numpy as np

# 能用到的API

In [1]:
import tensorflow as tf
tf.math.exp()
tf.math.log()
tf.reduce_sum()
tf.reduce_mean()

## 实现softmax函数

In [3]:
import numpy as np
print(tf.math.exp([1, 2, 3, 4.]))
print(np.exp([1, 2, 3, 4.]))

@tf.function
def func():
    print(tf.math.exp([1, 2, 3, 4.]))
func()

test_data = np.random.normal(size=[10, 5])
print(test_data)
print('--'*20)
print(tf.reduce_sum(test_data, axis=1, keepdims=True)) # [N, 1]

tf.Tensor([ 2.7182817  7.389056  20.085537  54.59815  ], shape=(4,), dtype=float32)
[ 2.71828183  7.3890561  20.08553692 54.59815003]
Tensor("Exp:0", shape=(4,), dtype=float32)
[[ 0.04869688  1.03508306  0.58859219 -0.13328246 -0.21204523]
 [ 0.7523976   0.28543778 -0.28624605 -1.14556942  0.22607593]
 [-0.24515495  1.2467467   1.40931426  1.83919203 -0.82693547]
 [-0.19342919 -0.23907839 -2.21973879 -1.32312583  2.36743738]
 [-0.22354752  0.08595743  1.99159433  1.06231891  0.41089149]
 [ 0.80098144  1.00561946 -0.71012586  0.08860742  0.1335414 ]
 [-0.32340475  0.73075424 -0.67997756 -0.52945451 -0.39506264]
 [-0.23538123 -0.06008378  1.94891277  0.08224466  1.83652073]
 [-0.76224826 -1.00813168 -0.46573321  0.57136874 -1.02536849]
 [-0.2479228   1.13761854 -0.44596084 -2.2305736   0.25281442]]
----------------------------------------
tf.Tensor(
[[ 1.32704444]
 [-0.16790416]
 [ 3.42316257]
 [-1.60793481]
 [ 3.32721464]
 [ 1.31862386]
 [-1.19714523]
 [ 3.57221315]
 [-2.69011291]
 [-1.

# softmax function: $\frac{e^{x_i}}{\sum_{j=1}^{d}{e^{x_j}}}$
# $\sum_{i}{\frac{e^{x_i}}{\sum_{j=1}^{d}{e^{x_j}}}} == 1$

In [4]:
def softmax(x):
    epsilon = 1e-12
    '''shape of x [N, d]'''
    x_exp = tf.math.exp(x) # shape=[N, d]
    denominator = tf.reduce_sum(x_exp, axis=1, keepdims=True) #shape=[N, 1]
    prob_x = x_exp/(denominator+epsilon)
    return prob_x

test_data = np.random.normal(size=[10, 5])
(softmax(test_data).numpy() - tf.nn.softmax(test_data, axis=-1).numpy())**2 <0.000001

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## 实现sigmoid函数

# $\frac{1}{1+e^{-x}}$

In [5]:
def sigmoid(x):
    '''shape=[N, d]'''
    x_exp = tf.math.exp(-x)
#     prob_x = 1/(1+x_exp)
    prob_x = tf.divide(1, tf.add(1, x_exp))
    return prob_x

test_data = np.random.normal(size=[10, 5])
(sigmoid(test_data).numpy() - tf.nn.sigmoid(test_data).numpy())**2 < 0.0001

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## 实现 softmax 交叉熵loss函数
# $\sum_{c}{-p(y=c|x)*log(p(\hat{y}=c|x))}$
# $s_{ij} = a_{ij}*b_{ij}$ 
# $s_{ik} = \sum_j{a_{ij}*b_{jk}}$ matmul

In [6]:
def softmax_ce(p, label):
    '''
        x shape = [N, c] ===> probability
        label shape = [N, c]
    '''
    loss = tf.reduce_mean(tf.reduce_sum(-label*tf.math.log(p), axis=1))
    return loss

test_data = np.random.normal(size=[10, 5])
prob = tf.nn.softmax(test_data)
label = np.zeros_like(test_data)
label[np.arange(10), np.random.randint(0, 5, size=10)]=1.

((tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(label, test_data))
  - softmax_ce(prob, label))**2 < 0.0001).numpy()


True

## 实现 sigmoid 交叉熵loss函数

# $-p(y=1|x)*log(p(\hat{y}=1|x)-p(y=0|x)*log(p(\hat{y}=0|x))$

In [7]:
def sigmoid_ce(p, label):
    epsilon = 1e-12
    '''shape = [N,]'''
    losses = -label * tf.math.log(p+epsilon) - (1.-label) * tf.math.log(1.-p+epsilon) # shape=[N,]
    loss = tf.reduce_mean(losses)
    return loss

test_data = np.random.normal(size=[10])
prob = tf.nn.sigmoid(test_data)
label = np.random.randint(0, 2, 10).astype(test_data.dtype)
print (label)

((tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(label, test_data))- sigmoid_ce(prob, label))**2 < 0.0001).numpy()


[1. 1. 1. 0. 1. 1. 1. 0. 1. 1.]


True

In [8]:
# @tf.function
def func(x):
    x_exp = tf.math.exp(x)
    out = x_exp+1
    return out

func(0.1)

<tf.Tensor: shape=(), dtype=float32, numpy=2.105171>

array([ 2.7182817,  7.389056 , 20.085537 , 54.59815  ], dtype=float32)